In [0]:
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
import pandas as pd

from EDA.ValidationData import Validation_data, DataTypeAnalysis
from EDA.StatisticalAnalysis import StatisticalAnalysis
from EDA.PlotGeometryAnalysis import PlotGeometryAnalysis

# Preprocesing Data

In [0]:
#1. Leemos los datos de PROCEESED la tabla Delta
df_delta2 = spark.read.format("delta").load("/mnt/datalakemlopsd4m/presentation/proyectocongestion_presentation/data_congestion_serietemp_balanceada")
datos = df_delta2.toPandas()

In [0]:
c_tarjet = ['congestion']
c_identidad = ['id_equipment', 'id_equipo', 'id_equipment_congestion', 'id_worker', 'id_trabaj_t', 'id_path', 'tramosidsnew_t']
c_categoria = ['n_sat', 'isload_t', 'marcha_t', 'alert_level', 'SERV_BRK_STAT']
c_coord = ['precisiongps_t', 'x', 'y', 'z', 'start_xcoorint', 'start_ycoorint', 'start_zcoorint', 'end_xcoorint', 'end_ycoorint', 'end_zcoorint', 'latitude_t', 'longitud_t', 'Lon_Coor', 'Lat_Coor', 'Z_Coor', 'Lat']

c_truck_llanta = ['tempeje1_t', 'tempeje2_t', 'tempeje3_t', 'tempeje4_t']
c_truck_mov = ['direccion_t', 'speed_t', 'pitch_t', 'roll_t', 'segment_angle_t']
c_truck_status = ['tonelaje_t', 'fuel_rate_t', 'combustibleint_t']

c_others2 = ['ENG_SPD', 'ENG_OIL_PRES',
       'ENG_COOL_TEMP', 'DIFF_LUBE_PRES',
       'DIFF_TEMP', 'AUTO_LUBE', 'GROUND_SPD', 'BOOST_PRES',
       'THROTTL_POS', 'AMB_AIR_TEMP', 'AIR_FLTR', 'RT_LT_EXH_TEMP',
       'LCKUP_SLIP', 'TRN_LUBE_TEMP', 'TRN_OUT_SPD', 'TC_OUT_TEMP', 'RETARDER', 'RETARDER_MODE',
       'BRK/AIR_PRES',
       'RTF_LTF_BRKTEMP', 'RTR_LTR_BRKTEMP', 'RT_F_BRK_TEMP',
       'RT_R_BRK_TEMP', 'LT_R_BRK_TEMP', 'PAYLOAD',
       'Tire_Press_N4', 'Tire_Press_N6', 'Hourmeter_MSPU', 'Speed_GPS',
       'Direction']

c_variables = ['id_equipo', 'id_worker', 'id_path', 'n_sat', 'isload_t', 'marcha_t', 'precisiongps_t', 'x', 'y', 'z', 'direccion_t', 'speed_t', 'pitch_t', 'roll_t', 'segment_angle_t', 'tonelaje_t', 'fuel_rate_t', 'combustibleint_t', 'LCKUP_SLIP', 'BRK/AIR_PRES', 'RTF_LTF_BRKTEMP', 'RTR_LTR_BRKTEMP', 'RT_F_BRK_TEMP', 'RT_R_BRK_TEMP', 'LT_R_BRK_TEMP', 'SERV_BRK_STAT', 'Tire_Press_N4', 'Tire_Press_N6', 'Hourmeter_MSPU', 'Direction']

## Columnas eliminar
## Sin presencia de registros C4M
c_drop1 = ['tempeje5_t', 'tempeje6_t', 'presllanta1_t',
       'presllanta2_t', 'presllanta3_t', 'presllanta4_t', 'presllanta5_t',
       'presllanta6_t', 'templlanta1_t',
       'templlanta2_t', 'templlanta3_t', 'templlanta4_t', 'templlanta5_t',
       'templlanta6_t', 'bateriasensorllanta1_t',
       'bateriasensorllanta2_t', 'bateriasensorllanta3_t',
       'bateriasensorllanta4_t', 'bateriasensorllanta5_t',
       'bateriasensorllanta6_t']
## Sin presencia de registros H4M
c_drop2 = ['SYS_VOLTAGE', 'HI_BOOST_PRES', 'LO_BOOST_PRES', 'BRK_STROKE']

## Columnas duplicadas
c_drop3 = ['eq_id', 'bearing_t', 'gear_t', 'Gear', 'frecuencia_t']


In [0]:
# 1. Filtramos datos atipicos
# 1.1. Coordenadas (0,0,0)
mask = (datos['x'] == 0) & (datos['y'] == 0) & (datos['z'] == 0)
datos = datos[~mask]

# 1.2. Filtramos equipos con mayor informacion
mask = (datos['eq_id'] == 61) | ((datos['eq_id'] == 199))
datos = datos[mask]

# 1.3. Filtramos datos con error de satelites
mask = (datos['n_sat'] > 0)
datos = datos[mask]

# 1.4. Filtramos rango de combustible
mask = (datos['combustibleint_t'] <= 100) & (datos['combustibleint_t'] >= 0)
datos = datos[mask]

# 1.5. Filtramos el porcentaje de combustible
mask = (datos['Hourmeter_MSPU'] <= 360)
datos = datos[mask]

# 1.6. Filtramos la temperatura diferencial del vehiculo
mask = (datos['DIFF_TEMP'] >= -100)
datos = datos[mask]

# 1.7. Filtramos porcentaje de lubricante
mask = (datos['AUTO_LUBE'] <= 1) & (datos['AUTO_LUBE'] >= 0)
datos = datos[mask]

# 1.8. Filtramos la temperatura del aire filtrado
mask = (datos['RT_LT_EXH_TEMP'] >= -100)
datos = datos[mask]

# 1.9. Filtramos la presion del lubricante del diferencial del vehiculo
mask = (datos['DIFF_LUBE_PRES'] >= -100)
datos = datos[mask]

# 1.10. Filtramos la temperatura del lubricante de la transmision
mask = (datos['TRN_LUBE_TEMP'] <= 101)
datos = datos[mask]

In [0]:
# 2. Eliminar columnas
# 2.1. Eliminar columnas del C4M y H4M
datos = datos.drop(columns=c_drop1+c_drop2+c_drop3)

In [0]:
datos.shape

In [0]:
for categoria in c_categoria:
    print(f'----- {categoria} --------')
    print(datos[categoria].unique())

In [0]:
mes = 3
dia = 27
mask = (datos['mes'] == mes) & (datos['dia'] == dia)
datos[mask][c_categoria].groupby(by=['id_equipment', 'id_worker']).count()

In [0]:
# Analisis tramos del camion
mes = 3
dia = 29
fig, ax = plt.subplots(1,2, figsize=(15,5))
mask = (datos['mes'] == mes) & (datos['dia'] == dia) #& (datos['id_equipment'] == 198.0)
sns.scatterplot(data=datos[mask], x='x', y='y', hue='id_equipment', palette='Dark2', ax=ax[0])
mask = (datos['mes'] == mes) & (datos['dia'] == dia) #& (datos['id_equipment'] == 199.0)
sns.scatterplot(data=datos[mask], x='x', y='y', hue='id_equipment', palette='Dark2', ax=ax[1])

In [0]:
# Analisis tramos del camion
mes = 3
dia = 29
fig, ax = plt.subplots(1,2, figsize=(15,5))
mask = (datos['mes'] == mes) & (datos['dia'] == dia) #& (datos['id_equipment'] == 198.0)
sns.scatterplot(data=datos[mask], x='x', y='y', hue='id_equipo', palette='Dark2', ax=ax[0])
mask = (datos['mes'] == mes) & (datos['dia'] == dia) #& (datos['id_equipment'] == 199.0)
sns.scatterplot(data=datos[mask], x='x', y='y', hue='id_equipo', palette='Dark2', ax=ax[1])

In [0]:
mask = (datos['mes'] == mes) & (datos['dia'] == dia) & (datos['id_equipment'] == 204.0)
data_mes_dia_equip = datos[mask].copy()

In [0]:
data_mes_dia_equip = data_mes_dia_equip.set_index('instant_date_t')
data_mes_dia_equip = data_mes_dia_equip.sort_index()
df_time_diffs = data_mes_dia_equip.index.to_series().diff().dt.total_seconds()


In [0]:
data_mes_dia_equip[df_time_diffs>2].index

In [0]:
set(df_time_diffs)

In [0]:
timeanalisis = pd.Timestamp('2024-03-27 10:18:26')
timedelta = pd.Timedelta(seconds=173)
mask = (data_mes_dia_equip.index < timeanalisis + timedelta) & (data_mes_dia_equip.index > timeanalisis - timedelta)
data_mes_dia_equip[mask]

In [0]:
timeanalisis_inf = pd.Timestamp('2024-03-27 10:15:34')
timeanalisis_sup = pd.Timestamp('2024-03-27 10:18:30')
timedelta = pd.Timedelta(seconds=200)
mask = (data_mes_dia_equip.index < timeanalisis_sup + timedelta) & (data_mes_dia_equip.index > timeanalisis_inf - timedelta)
data_mes_dia_equip[mask]#['x'].diff()

In [0]:
sns.scatterplot(data=data_mes_dia_equip[mask], x='x', y='y', hue='id_equipment', palette='Dark2')
ax[1].set_title(data_mes_dia_equip[mask].shape)

In [0]:
#3. Eliminamos o Filtramos las filas donde la diferencia es distinta de cero (Con ello eliminamos las filas o registros de fechas duplicadas)
data_mes_dia_equip = data_mes_dia_equip[df_time_diffs != 0]

#4. # Reinterpolar el dataset con una periosidad en especifico
# 'S' o 'seg' : Segundo, 'T' o 'min': Minuto,'H' o 'Hr': Hora,'D': Día,'W': Semana,'M': Mes,'Q': Trimestre,'Y': Año
data_mes_dia_equip = data_mes_dia_equip.asfreq(freq='6S', method='bfill')

data_mes_dia_equip = data_mes_dia_equip.reset_index()

In [0]:
# Analisis tramos del camion
fig, ax = plt.subplots(1,2, figsize=(15,5))
mask = (datos['mes'] == mes) & (datos['dia'] == dia) & (datos['id_equipment'] == 204.0)
sns.scatterplot(data=datos[mask], x='x', y='y', hue='id_equipment', palette='Dark2', ax=ax[0])
ax[0].set_title(datos[mask].shape)
sns.scatterplot(data=data_mes_dia_equip, x='x', y='y', hue='id_equipment', palette='Dark2', ax=ax[1])
ax[1].set_title(data_mes_dia_equip.shape)

In [0]:
datos[mask]#.head()

In [0]:
data_mes_dia_equip#.head()

In [0]:
#Hacer el Balanceo de datos import pandas as pd
import numpy as np
from imblearn.over_sampling import SMOTE
 
# Convertimos a index la fecha
datos = datos.set_index('instant_date_t')

# Eliminamos registros con coordenadas (0,0,0)
mask = (datos['x']>0) & (datos['y']>0) & (datos['z']>0)
datos = datos[mask]

# Eliminacion de variables tipo object
datos = datos.drop(columns=['nombre', 'nombre_equipo'])


# Filtrar las fechas donde congestion == 1
fechas_congestion_1 = datos[datos['congestion'] == 1].index
 
# Generar un rango de fechas completo dentro del rango del DataFrame
fecha_min = datos.index.min()
fecha_max = datos.index.max()
rango_completo = pd.date_range(start=fecha_min, end=fecha_max, freq='S')
 
# Excluir las fechas de congestion == 1 del rango completo
fechas_disponibles = rango_completo.difference(fechas_congestion_1)
 
# Aplicar SMOTE para generar nuevas muestras de la clase minoritaria (congestion == 0)
smote = SMOTE(sampling_strategy='auto', random_state=42)
 
# Usar todas las columnas como características para SMOTE excepto 'congestion'
X = datos.drop(columns=['congestion'])
y = datos['congestion']
 
# Convertir las fechas del índice a números para usarlas con SMOTE
X['instant_date_num'] = X.index.astype(int) / 10**9  # Convertir a segundos
 
# Aplicar SMOTE
X_res, y_res = smote.fit_resample(X, y)
 
# Convertir de nuevo a DataFrame
X_res = pd.DataFrame(X_res, columns=X.columns)
y_res = pd.Series(y_res, name='congestion')
 
# Filtrar las nuevas muestras generadas por SMOTE (la diferencia entre las originales y las nuevas)
nuevas_muestras = X_res[len(datos):]
nuevas_congestion = y_res[len(datos):]
 
# Asignar nuevas fechas secuenciales de fechas_disponibles a las nuevas muestras
nuevas_fechas_asignadas = fechas_disponibles[:len(nuevas_muestras)]
nuevas_muestras['instant_date'] = nuevas_fechas_asignadas
 
# Crear el DataFrame final combinando los datos originales y las nuevas muestras
nuevas_muestras = nuevas_muestras.set_index('instant_date')
nuevas_muestras = nuevas_muestras.drop(columns=['instant_date_num'])
nuevas_muestras['congestion'] = nuevas_congestion.values
 
# Combinar los datos originales y las nuevas muestras
datos_cong_balanceado = pd.concat([datos, nuevas_muestras]).sort_index()

datos_cong_balanceado.index.name = 'instant_date_t'
#datos_cong_balanceado = datos_cong_balanceado.reset_index()

In [0]:
mask = (datos_cong_balanceado['x'] == 0) & (datos_cong_balanceado['y'] == 0) & (datos_cong_balanceado['z'] == 0)
datos_cong_balanceado[mask]

In [0]:
# 3. Convertir el DataFrame de Pandas a un DataFrame de Spark
spark_datos = spark.createDataFrame(datos_cong_balanceado)
 
# 4. Guardar los datos preprocesados en una tabla Delta en el Azure Storage
# Asegurar que las columnas de fecha y hora mantengan sus tipos de datos
#spark_datos = spark_datos.withColumn("Event_Date", col("Event_Date").cast("timestamp"))
#spark_datos = spark_datos.withColumn("instant_date_t", col("instant_date_t").cast("timestamp"))
 
# Nombre de la tabla Delta a guardar
nombre_tabla_delta = "dbproyectocongestion_presentation.tablacaracteristicas_congestion_tabladelta_2"
 
# 4.1 Verificar si ya existe la tabla Delta
if spark.catalog.tableExists(nombre_tabla_delta):
    # Eliminar la tabla Delta existente
    spark.sql("DROP TABLE IF EXISTS " + nombre_tabla_delta)
 
# 4.2 Guardar los datos preprocesados en una tabla Delta
spark_datos.write.format("delta").mode("overwrite").option("mergeSchema", "true").saveAsTable(nombre_tabla_delta)

Leer Datos Sin Balancear

In [0]:
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
import numpy as np
import pandas as pd
from imblearn.over_sampling import SMOTE

from EDA.ValidationData import Validation_data, DataTypeAnalysis
from EDA.StatisticalAnalysis import StatisticalAnalysis
from EDA.PlotGeometryAnalysis import PlotGeometryAnalysis

#1. Leemos los datos de PROCEESED la tabla Delta
# /mnt/datalakemlopsd4m/processed/proyectocongestion_processed/datapreprocessed_congestion_tabladelta
df_delta = spark.read.format("delta").load("/mnt/datalakemlopsd4m/presentation/proyectocongestion_presentation/data_congestion_serietemp_balanceada")
datos_sin_balanceo = df_delta.toPandas()

# 2. Limpieza de variables No prioritarias
columnas_a_eliminar = ['nombre_equipo','nombre', 'nombre_equipo','nombre','start_time_alert','end_time_alert','Event_Date']

# 2.1 Filtrar las columnas que existen en el DataFrame
columnas_existentes = [col for col in columnas_a_eliminar if col in datos_sin_balanceo.columns]

# 2.2 Verificar si hay columnas para eliminar
if columnas_existentes:
    datos_sin_balanceo.drop(columnas_existentes, axis=1, inplace=True)


# Filtramos los valores de x, y, z que son 0s
mask = (datos_sin_balanceo['x'] == 0) & (datos_sin_balanceo['y'] == 0) & (datos_sin_balanceo['z'] == 0)
datos_sin_balanceo = datos_sin_balanceo[~mask]

# 3.2. Filtramos equipos con mayor informacion
mask = (datos_sin_balanceo['id_equipo'] == 61) | ((datos_sin_balanceo['id_equipo'] == 199))
datos_sin_balanceo = datos_sin_balanceo[mask]

# 3.3. Filtramos datos con error de satelites
mask = (datos_sin_balanceo['n_sat'] > 0)
datos_sin_balanceo = datos_sin_balanceo[mask]

# 3.4. Filtramos rango de combustible
mask = (datos_sin_balanceo['combustibleint_t'] <= 100) & (datos_sin_balanceo['combustibleint_t'] >= 0)
datos_sin_balanceo = datos_sin_balanceo[mask]

# 3.5. Filtramos el porcentaje de combustible
mask = (datos_sin_balanceo['Hourmeter_MSPU'] <= 360)
datos_sin_balanceo = datos_sin_balanceo[mask]

# 3.6. Filtramos la temperatura diferencial del vehiculo
mask = (datos_sin_balanceo['DIFF_TEMP'] >= -100)
datos_sin_balanceo = datos_sin_balanceo[mask]

# 3.7. Filtramos porcentaje de lubricante
mask = (datos_sin_balanceo['AUTO_LUBE'] <= 1) & (datos_sin_balanceo['AUTO_LUBE'] >= 0)
datos_sin_balanceo = datos_sin_balanceo[mask]

# 3.8. Filtramos la temperatura del aire filtrado
mask = (datos_sin_balanceo['RT_LT_EXH_TEMP'] >= -100)
datos_sin_balanceo = datos_sin_balanceo[mask]

# 3.9. Filtramos la presion del lubricante del diferencial del vehiculo
mask = (datos_sin_balanceo['DIFF_LUBE_PRES'] >= -100)
datos_sin_balanceo = datos_sin_balanceo[mask]

# 3.10. Filtramos la temperatura del lubricante de la transmision
mask = (datos_sin_balanceo['TRN_LUBE_TEMP'] <= 101)
datos_sin_balanceo = datos_sin_balanceo[mask]


datos = datos_sin_balanceo.copy()


# Convertimos a index la fecha
datos = datos.set_index('instant_date_t')

datos = datos.sort_index()

#2.4 Identificamos la periocidad de la serie temporal
df_time_diffs = datos.index.to_series().diff().dt.total_seconds()
 
# # 2.4.1 Contar cuántas diferencias de tiempo tienen cada valor específico
# diferencias_frecuencias = df_time_diffs.value_counts().sort_index()
# # 2.4.2 Mostrar los recuentos
# print(diferencias_frecuencias)
 
#3. Eliminamos o Filtramos las filas donde la diferencia es distinta de cero (Con ello eliminamos las filas o registros de fechas duplicadas)
datos = datos[df_time_diffs != 0]


In [0]:
#4. # Reinterpolar el dataset con una periosidad en especifico
# 'S' o 'seg' : Segundo, 'T' o 'min': Minuto,'H' o 'Hr': Hora,'D': Día,'W': Semana,'M': Mes,'Q': Trimestre,'Y': Año
datos = datos.asfreq(freq='6S', method='bfill')

In [0]:
datos.shape

In [0]:

# Filtrar las fechas donde congestion == 1
fechas_congestion_1 = datos[datos['congestion'] == 1].index

# Generar un rango de fechas completo dentro del rango del DataFrame
fecha_min = datos.index.min()
fecha_max = datos.index.max()
rango_completo = pd.date_range(start=fecha_min, end=fecha_max, freq='S')

# Excluir las fechas de congestion == 1 del rango completo
fechas_disponibles = rango_completo.difference(fechas_congestion_1)

# Aplicar SMOTE para generar nuevas muestras de la clase minoritaria (congestion == 0)
smote = SMOTE(sampling_strategy='auto', random_state=42)

# Usar todas las columnas como características para SMOTE excepto 'congestion'
X = datos.drop(columns=['congestion'])
y = datos['congestion']

# Convertir las fechas del índice a números para usarlas con SMOTE
X['instant_date_num'] = X.index.astype(int) / 10**9  # Convertir a segundos

# Aplicar SMOTE
X_res, y_res = smote.fit_resample(X, y)

# Convertir de nuevo a DataFrame
X_res = pd.DataFrame(X_res, columns=X.columns)
y_res = pd.Series(y_res, name='congestion')

# Filtrar las nuevas muestras generadas por SMOTE (la diferencia entre las originales y las nuevas)
nuevas_muestras = X_res[len(datos):]
nuevas_congestion = y_res[len(datos):]

# Asignar nuevas fechas secuenciales de fechas_disponibles a las nuevas muestras
nuevas_fechas_asignadas = fechas_disponibles[:len(nuevas_muestras)]
nuevas_muestras['instant_date'] = nuevas_fechas_asignadas

# Crear el DataFrame final combinando los datos originales y las nuevas muestras
nuevas_muestras = nuevas_muestras.set_index('instant_date')
nuevas_muestras = nuevas_muestras.drop(columns=['instant_date_num'])
nuevas_muestras['congestion'] = nuevas_congestion.values

# Combinar los datos originales y las nuevas muestras
datos_balanceados = pd.concat([datos, nuevas_muestras]).sort_index()


datos_balanceados = datos_balanceados.reset_index(names='instant_date_t')

# 3. Convertir el DataFrame de Pandas a un DataFrame de Spark
spark_datos = spark.createDataFrame(datos_balanceados)
 
# 4. Guardar los datos preprocesados en una tabla Delta en el Azure Storage
# Asegurar que las columnas de fecha y hora mantengan sus tipos de datos
#spark_datos = spark_datos.withColumn("Event_Date", col("Event_Date").cast("timestamp"))
#spark_datos = spark_datos.withColumn("instant_date_t", col("instant_date_t").cast("timestamp"))
 
# Nombre de la tabla Delta a guardar
nombre_tabla_delta = "dbproyectocongestion_presentation.tablacaracteristicas_congestion_tabladelta_v3"
 
# 4.1 Verificar si ya existe la tabla Delta
if spark.catalog.tableExists(nombre_tabla_delta):
    # Eliminar la tabla Delta existente
    spark.sql("DROP TABLE IF EXISTS " + nombre_tabla_delta)
 
# 4.2 Guardar los datos preprocesados en una tabla Delta
spark_datos.write.format("delta").mode("overwrite").option("mergeSchema", "true").saveAsTable(nombre_tabla_delta)

In [0]:
datos_balanceados.shape

In [0]:
#1. Leemos los datos de PROCEESED la tabla Delta
df_delta2 = spark.read.format("delta").load("/mnt/datalakemlopsd4m/presentation/proyectocongestion_presentation/tablacaracteristicas_congestion_tabladelta_v3")
datos = df_delta2.toPandas()
datos.head()

In [0]:
datos.head()

In [0]:
datos['congestion'].value_counts()

In [0]:
datos['n_sat'].unique()

In [0]:
# Convertimos a index la fecha
datos = datos.set_index('instant_date_t')
#2.4 Identificamos la periocidad de la serie temporal
df_time_diffs = datos.index.to_series().diff().dt.total_seconds()
 
# # 2.4.1 Contar cuántas diferencias de tiempo tienen cada valor específico
diferencias_frecuencias = df_time_diffs.value_counts().sort_index()
# # 2.4.2 Mostrar los recuentos
print(diferencias_frecuencias)

In [0]:
datos['x'].unique()

In [0]:
# #2. Ajuste de los datos a Series Temporales
# # #2.1 Se establece la columna 'Time' como el índice del DataFrame "datos"
# datos = datos.set_index('instant_date_t')

# #2.3 Ordenamos el dataset de forma ascendente segun el datetime
datos.sort_index(inplace=True)

#2.4 Identificamos la periocidad de la serie temporal
df_time_diffs = datos.index.to_series().diff().dt.total_seconds()

# # 2.4.1 Contar cuántas diferencias de tiempo tienen cada valor específico
# diferencias_frecuencias = df_time_diffs.value_counts().sort_index()
# # 2.4.2 Mostrar los recuentos
# print(diferencias_frecuencias)

#3. Eliminamos o Filtramos las filas donde la diferencia es distinta de cero (Con ello eliminamos las filas o registros de fechas duplicadas)
datos = datos[df_time_diffs != 0]

#4. # Reinterpolar el dataset con una periosidad en especifico
# 'S' o 'seg' : Segundo, 'T' o 'min': Minuto,'H' o 'Hr': Hora,'D': Día,'W': Semana,'M': Mes,'Q': Trimestre,'Y': Año
#datos_cong_balanceado = datos_cong_balanceado.asfreq(freq='5T', method='bfill')
datos = datos.asfreq(freq='6s', method='bfill')

# Filtramos los valores de x, y, z que son 0s
mask = (datos['x'] == 0) & (datos['y'] == 0) & (datos['z'] == 0)
datos = datos[~mask]

datos.head()